In [43]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import math
from timeit import default_timer as timer
from datetime import datetime, timedelta
import numba
from ensemble_processing import load_data, load, save
from tsfresh import extract_relevant_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from tsfresh.utilities.dataframe_functions import impute

In [2]:
pd.options.display.max_rows=500

In [44]:
# experiment_symbols = ['WAX', 'IVV', 'WESN', 'IAA', 'XRO', 'MTB', 'MXR', 'RCP', 'ISX', 'IMI']
experiment_symbols = ['WAX']

In [46]:
all_df = pd.read_pickle('../data/ml-20180714-data.pkl.gz', compression='gzip')
all_df.head(100)

,symbol,quoteDate,lastTradePriceOnly,adjustedPrice,volume,daysHigh,daysLow,previousClose,change,changeInPercent,...,Future12WeekReturn,Future12WeekRiskAdjustedReturn,Future26WeekDividend,Future26WeekPrice,Future26WeekReturn,Future26WeekRiskAdjustedReturn,Future52WeekDividend,Future52WeekPrice,Future52WeekReturn,Future52WeekRiskAdjustedReturn
0,CG1,2018-05-14,0.43000,0.430000,27759,0.43000,0.43000,0.43000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CG1,2018-05-15,0.42500,0.425000,45633,0.43000,0.42500,0.43000,-0.00500,-0.011628,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CG1,2018-05-17,0.42000,0.420000,287372,0.43000,0.41500,0.42500,-0.00500,-0.011765,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CG1,2018-05-18,0.46500,0.465000,42373,0.46500,0.42000,0.42000,0.04500,0.107143,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CG1,2018-05-21,0.51000,0.510000,229346,0.51000,0.47000,0.46500,0.04500,0.096774,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,CG1,2018-05-22,0.50000,0.500000,2378,0.50000,0.50000,0.51000,-0.01000,-0.019608,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,CG1,2018-05-23,0.49000,0.490000,92622,0.50000,0.49000,0.50000,-0.01000,-0.020000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,CG1,2018-05-24,0.48500,0.485000,2922,0.48500,0.48500,0.49000,-0.00500,-0.010204,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,CG1,2018-05-25,0.47000,0.470000,20404,0.47000,0.46500,0.48500,-0.01500,-0.030928,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,CG1,2018-05-28,0.46500,0.465000,8805,0.46500,0.46500,0.47000,-0.00500,-0.010638,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
train_x_dfs = []

for symbol in experiment_symbols:
    # Filter to model data for this symbol and re-set the pandas indexes
    model_data = all_df.loc[all_df['symbol'] == symbol]

    print('Symbol:', symbol, 'number of records:', len(model_data))
    train_x_dfs.append(model_data)

# Create concatenated dataframe with all data
print('Creating concatenated dataframe')
df_train_x = pd.concat(train_x_dfs)

del train_x_dfs
del all_df

Symbol: WAX number of records: 2294
Creating concatenated dataframe


In [8]:
df_train_x.describe()

,lastTradePriceOnly,adjustedPrice,volume,daysHigh,daysLow,previousClose,change,changeInPercent,52WeekHigh,52WeekLow,...,Future12WeekReturn,Future12WeekRiskAdjustedReturn,Future26WeekDividend,Future26WeekPrice,Future26WeekReturn,Future26WeekRiskAdjustedReturn,Future52WeekDividend,Future52WeekPrice,Future52WeekReturn,Future52WeekRiskAdjustedReturn
count,18275.000000,18229.000000,1.827500e+04,18273.000000,18273.000000,18275.000000,17932.000000,17950.000000,18275.000000,18273.000000,...,16543.000000,16543.000000,15936.000000,15936.000000,15936.000000,15936.000000,14882.000000,14882.000000,14882.000000,14882.000000
mean,38.888290,37.297523,6.164693e+05,39.083229,38.725643,38.908596,-0.019991,0.288922,40.229813,30.857351,...,1.608522,-347.826843,0.240974,38.880257,1.783825,-693.938049,0.520500,40.226662,7.269506,-1078.868652
std,73.050064,71.151283,3.872008e+06,73.286949,72.839081,73.070587,0.586617,0.462492,74.174255,59.839161,...,74.247093,1415.953491,0.565421,72.971367,87.106598,2999.984619,1.116301,74.804466,145.029449,4683.137695
min,0.000600,0.000930,2.000000e+00,0.000600,0.000600,0.000600,-20.100006,-1.000000,0.002000,0.000000,...,-88.571426,-19250.304688,0.000000,0.000930,-93.333336,-36588.335938,0.000000,0.000930,-95.002502,-53526.023438
25%,0.011000,0.064170,1.140000e+04,0.011570,0.010790,0.011000,-0.010000,-0.005428,0.231000,0.042000,...,-14.301637,-52.626282,0.000000,0.063000,-27.028788,-73.209957,0.000000,0.035000,-49.997913,-139.240852
50%,0.845000,0.839000,5.810000e+04,0.850000,0.840000,0.840000,0.000000,0.005291,1.184200,0.573400,...,0.000000,0.000000,0.000000,0.786200,0.000000,0.000000,0.000000,0.641670,0.000000,0.000000
75%,42.689999,39.520000,2.412500e+05,42.939999,42.490002,42.700001,0.005000,1.000000,42.200001,33.900002,...,8.481688,0.106535,0.050000,40.349998,13.922661,0.084489,0.117857,41.307501,24.407178,0.094072
max,364.299988,364.299988,1.825003e+08,364.480011,363.299988,364.299988,13.290009,2.000000,364.480011,305.790009,...,4233.333496,1913.516724,4.458210,359.850006,3400.000000,1340.731079,6.167980,359.850006,4900.000000,4466.275391


In [59]:
df_time_series = pd.DataFrame()
df_time_series['symbol'] = df_train_x['symbol']
df_time_series['quoteDate'] = df_train_x['quoteDate']
df_time_series['Future8WeekReturn'] = df_train_x['Future8WeekReturn']
df_time_series.dropna(axis=0, how='any', inplace=True)
df_time_series.set_index('quoteDate', inplace=True)
df_time_series

,symbol,Future8WeekReturn
quoteDate,,
2007-07-02,WAX,-5.080055
2007-07-03,WAX,-5.537912
2007-07-04,WAX,-5.080055
2007-07-05,WAX,-3.380115
2007-07-06,WAX,-3.814983
2007-07-09,WAX,-4.664954
2007-07-10,WAX,-4.625984
2007-07-11,WAX,-2.990431
2007-07-12,WAX,-2.549911


In [63]:
df_shift, y = make_forecasting_frame(df_time_series["Future8WeekReturn"], kind="return", 
                                     max_timeshift=40, rolling_direction=1)

/Users/neilkloot/anaconda/lib/python3.6/site-packages/tsfresh/utilities/dataframe_functions.py:394: UserWarning: Your time stamps are not uniformly sampled, which makes rolling nonsensical in some domains.
  warnings.warn("Your time stamps are not uniformly sampled, which makes rolling "


In [64]:
df_shift

,time,value,id,kind
80145,2007-07-02,-5.080055,2007-07-03,return
78071,2007-07-02,-5.080055,2007-07-04,return
80146,2007-07-03,-5.537912,2007-07-04,return
75998,2007-07-02,-5.080055,2007-07-05,return
78072,2007-07-03,-5.537912,2007-07-05,return
80147,2007-07-04,-5.080055,2007-07-05,return
73926,2007-07-02,-5.080055,2007-07-06,return
75999,2007-07-03,-5.537912,2007-07-06,return
78073,2007-07-04,-5.080055,2007-07-06,return
80148,2007-07-05,-3.380115,2007-07-06,return


In [65]:
y

quoteDate
2007-07-03    -5.537912
2007-07-04    -5.080055
2007-07-05    -3.380115
2007-07-06    -3.814983
2007-07-09    -4.664954
2007-07-10    -4.625984
2007-07-11    -2.990431
2007-07-12    -2.549911
2007-07-13    -3.814983
2007-07-16    -3.814983
2007-07-17    -2.990431
2007-07-18    -4.267567
2007-07-19    -4.625984
2007-07-20    -4.684399
2007-07-23    -3.830885
2007-07-24    -4.684399
2007-07-25    -4.230085
2007-07-26    -2.990431
2007-07-27    -3.409091
2007-07-30    -1.292407
2007-07-31    -1.286950
2007-08-01    -2.160743
2007-08-02    -2.573899
2007-08-03    -2.151619
2007-08-06    -0.851927
2007-08-07    -0.851927
2007-08-08    -0.851927
2007-08-09    -1.709230
2007-08-10     0.000000
2007-08-13     0.872211
2007-08-14     1.303728
2007-08-15     3.095975
2007-08-16     9.719222
2007-08-17     7.718405
2007-08-20    15.566773
2007-08-21    15.052163
2007-08-22    14.585666
2007-08-23    12.043541
2007-08-24    13.088044
2007-08-27    10.680133
2007-08-28    11.680185
2007-0

In [66]:
X = extract_relevant_features(df_shift, y, column_id="id", column_sort="time", column_value="value", 
                              # impute_function=impute, 
                              ml_task='regression', show_warnings=True)

Feature Extraction:   0%|          | 0/20 [00:00<?, ?it/s]/Users/neilkloot/anaconda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/neilkloot/anaconda/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/Users/neilkloot/anaconda/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1453: RuntimeWarning: invalid value encountered in true_divide
  similarity_ratio = A / B
/Users/neilkloot/anaconda/lib/python3.6/site-packages/tsfresh/feature_extraction/feature_calculators.py:1496: RuntimeWarning: invalid value encountered in true_divide
  return sum_product / ((len(x) - lag) * np.var(x))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/signaltools.py:375: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In fut

/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:1353: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 11, using nperseg = 11
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 12, using nperseg = 12
  .format(nperseg, inp

/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 26, using nperseg = 26
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_leng

/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 38, using nperseg = 38
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_leng

/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_leng

...

/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_leng

/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_leng

/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_leng

/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_leng

/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
Feature Extraction:  95%|█████████▌| 19/20 [01:13<00:03,  3.87s/it]/Users/neilkloot/anaconda/lib/python3.6/site-packages/scipy/signal/spectral.py:1633: UserWarning: nperseg = 256 is greater than input length  = 40, using nperseg = 40
  .format(nperseg, input_length))
Feature Extraction: 100%|██████████| 20/20 [01:13<00:00,  3.70s/it]
 'value__agg_linear_trend__f_agg_"max"__chunk_len_50__attr_"r

In [67]:
X

variable,value__ar_coefficient__k_10__coeff_0,"value__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.0",value__mean_change,value__time_reversal_asymmetry_statistic__lag_3,value__time_reversal_asymmetry_statistic__lag_1,"value__agg_linear_trend__f_agg_""min""__chunk_len_5__attr_""slope""",value__time_reversal_asymmetry_statistic__lag_2,"value__agg_linear_trend__f_agg_""mean""__chunk_len_5__attr_""slope""","value__agg_linear_trend__f_agg_""mean""__chunk_len_10__attr_""slope""","value__linear_trend__attr_""slope""",...,"value__augmented_dickey_fuller__attr_""pvalue""","value__augmented_dickey_fuller__attr_""teststat""",value__longest_strike_above_mean,"value__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_9__w_10","value__change_quantiles__f_agg_""var""__isabs_True__qh_0.4__ql_0.2",value__has_duplicate_max,value__autocorrelation__lag_7,"value__fft_coefficient__coeff_11__attr_""abs""","value__change_quantiles__f_agg_""mean""__isabs_True__qh_0.2__ql_0.0","value__agg_linear_trend__f_agg_""mean""__chunk_len_5__attr_""intercept"""
id,,,,,,,,,,,,,,,,,,,,,
2007-07-03,0.678590,0.000000,0.009044,0.000000,0.000000,0.039158,0.000000,0.062590,0.111510,0.014690,...,3.760395e-01,-1.808926,1.0,8.968632,0.000000e+00,0.0,0.243004,9.076981,0.000000,3.751022
2007-07-04,0.678590,-0.457857,-0.457857,0.000000,0.000000,0.039158,0.000000,0.062590,0.111510,-0.457857,...,3.760395e-01,-1.808926,1.0,8.968632,0.000000e+00,0.0,0.243004,9.076981,0.000000,3.751022
2007-07-05,0.678590,0.000000,0.000000,0.000000,0.000000,0.039158,0.000000,0.062590,0.111510,0.000000,...,3.760395e-01,-1.808926,1.0,8.968632,0.000000e+00,1.0,0.243004,9.076981,0.000000,3.751022
2007-07-06,0.678590,0.566647,0.566647,0.000000,48.878498,0.039158,0.000000,0.062590,0.111510,0.555768,...,3.760395e-01,-1.808926,1.0,8.968632,0.000000e+00,0.0,0.243004,9.076981,0.000000,3.751022
2007-07-09,0.678590,0.316268,0.316268,0.000000,45.264324,0.039158,57.165169,0.062590,0.111510,0.468794,...,3.760395e-01,-1.808926,2.0,8.968632,0.000000e+00,0.0,0.243004,9.076981,0.000000,3.751022
2007-07-10,0.678590,0.083020,0.083020,0.000000,24.089735,0.872958,43.635376,-0.086330,0.111510,0.255550,...,3.760395e-01,-1.808926,2.0,8.968632,0.000000e+00,0.0,0.243004,9.076981,0.457857,-4.578624
2007-07-11,0.678590,0.075679,0.075679,14.896950,12.884875,0.872958,34.694756,-0.066845,0.111510,0.156186,...,3.760395e-01,-1.808926,2.0,8.968632,0.000000e+00,0.0,0.243004,9.076981,0.457857,-4.578624
2007-07-12,0.678590,0.298518,0.298518,48.890240,20.620901,0.872958,28.916252,0.484834,0.111510,0.238065,...,3.760395e-01,-1.808926,2.0,8.968632,0.000000e+00,0.0,-1.346206,9.076981,0.457857,-4.578624
2007-07-13,0.678590,0.316268,0.316268,62.612339,24.039415,0.872958,30.582733,0.870804,0.111510,0.289772,...,7.869246e-01,-0.903176,2.0,8.968632,0.000000e+00,0.0,-1.687007,9.076981,0.457857,-4.578624


In [24]:
df_time_series[df_time_series.isnull().any(axis=1)]

,symbol,quoteDate,adjustedPrice


In [37]:
y = pd.DataFrame(data={
    'adjustedPrice': df_time_series['adjustedPrice'].values
    },
                 index=df_time_series[['symbol', 'quoteDate']])
y['adjustedPrice']

(WAX, 2007-07-02 00:00:00)    0.5059
(WAX, 2007-07-03 00:00:00)    0.5038
(WAX, 2007-07-04 00:00:00)    0.5059
(WAX, 2007-07-05 00:00:00)    0.5059
(WAX, 2007-07-06 00:00:00)    0.5059
(WAX, 2007-07-09 00:00:00)    0.5059
(WAX, 2007-07-10 00:00:00)    0.5080
(WAX, 2007-07-11 00:00:00)    0.5016
(WAX, 2007-07-12 00:00:00)    0.5059
(WAX, 2007-07-13 00:00:00)    0.5059
(WAX, 2007-07-16 00:00:00)    0.5059
(WAX, 2007-07-17 00:00:00)    0.5016
(WAX, 2007-07-18 00:00:00)    0.5038
(WAX, 2007-07-19 00:00:00)    0.5080
(WAX, 2007-07-20 00:00:00)    0.5038
(WAX, 2007-07-23 00:00:00)    0.5038
(WAX, 2007-07-24 00:00:00)    0.5038
(WAX, 2007-07-25 00:00:00)    0.5059
(WAX, 2007-07-26 00:00:00)    0.5016
(WAX, 2007-07-27 00:00:00)    0.5016
(WAX, 2007-07-30 00:00:00)    0.4952
(WAX, 2007-07-31 00:00:00)    0.4973
(WAX, 2007-08-01 00:00:00)    0.4952
(WAX, 2007-08-02 00:00:00)    0.4973
(WAX, 2007-08-03 00:00:00)    0.4973
(WAX, 2007-08-06 00:00:00)    0.4930
(WAX, 2007-08-07 00:00:00)    0.4930
(

In [ ]:
X = extract_relevant_features(df_time_series, y['adjustedPrice'], 
                              column_id='symbol', column_sort='quoteDate', show_warnings=True)